# Data Processsing

In [1]:
train_dir = '/home/ec2-user/SageMaker/train/'
test_dir = '/home/ec2-user/SageMaker/test/'
files_dir = '/home/ec2-user/SageMaker/'

## Labels

In [2]:
import json

with open(files_dir + '/label_num_to_disease_map.json', 'r') as f:
    labels_info = json.load(f)

NUM_LABELS = len(labels_info)
print(labels_info)

{'0': 'Cassava Bacterial Blight (CBB)', '1': 'Cassava Brown Streak Disease (CBSD)', '2': 'Cassava Green Mottle (CGM)', '3': 'Cassava Mosaic Disease (CMD)', '4': 'Healthy'}


## Load Data with Pytorch

In [3]:
import pandas as pd
import os
import torch
from torch.utils.data import Dataset, DataLoader

from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from PIL import Image
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv(files_dir + '/train.csv')

# Split the data into train, validation, and test sets with a ratio of 80:10:10
train_df, val_test_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(val_test_df, test_size=0.5, random_state=42)

In [5]:
class CassavaDataset(Dataset):
    def __init__(self, data, root_dir, transform=None):
        self.image_labels = data
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.image_labels)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.image_labels.iloc[index, 0])
        image = Image.open(img_path).convert('RGB')
        label = self.image_labels.iloc[index, 1]
        if self.transform:
            image = self.transform(image)
        return image, label

# Useful Functions

In [6]:
# Save the model so that we don't need to train the same model multiple times
def save_model(model, model_name):
  torch.save(model.state_dict(),  files_dir + 'parameter_search_models/' + model_name + '.pth')

# Training with Resnet with parameter search

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("We're using:", device)

We're using: cuda


In [8]:
# Train and validate function
def train_and_validate(model, train_loader, valid_loader, test_loader, optimizer, criterion):
    # return train accuracy, validation accuracy, test accuracy
    
    EPOCHS = 5 # Each epoch takes around 16 minutes
    
    train_loss, validation_loss = [], []
    train_acc, validation_acc = [], []

    for epoch in range(EPOCHS):
        # print("Training Epoch: ", epoch)
        batch_num = 0

        model.train()
        running_loss = 0.
        correct, total = 0, 0 

        for inputs, labels in train_loader:
            batch_num += 1
            # print("Current Batch: ", batch_num)
            # Forward pass
            inputs = inputs.to(device)
            labels = labels.to(device)

            predictions = model(inputs)
            loss = criterion(predictions, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            _, predicted = torch.max(predictions, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_loss.append(running_loss / len(train_loader))
        train_acc.append(correct/total)

        model.eval()
        running_loss = 0.
        correct, total = 0, 0 

        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            predictions = model(inputs)

            loss = criterion(predictions, labels)

            running_loss += loss.item()

            _, predicted = torch.max(predictions, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        validation_loss.append(running_loss / len(val_loader))
        validation_acc.append(correct/total)
        
    
    
    # Calculate the test accuracy 
    import numpy as np
    model.eval()

    test_predictions = np.array([])

    total = 0
    correct = 0

    for inputs, labels in test_loader:

        inputs = inputs.to(device)
        labels = labels.to(device)

        predictions = model(inputs)

        _, predicted = torch.max(predictions, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    return round(train_acc[4],3), round(validation_acc[4],3), round(correct/total,3)
    
    

In [9]:
from torchvision import models # resnet50, ResNet50_Weights

network_depths = [50]# [18, 34, 50]  # ResNet-18, ResNet-34, and ResNet-50
batch_sizes = [64] # [32, 64, 128]
optimizers = ['adam']# ['adam', 'SGD']
learning_rates = [0.001]# [1e-2, 1e-3, 1e-4]

# Store accuracies for each hyperparameter combination
hyperparams_accuracies = {}

In [ ]:
output = open("parameter_search_result.txt", 'w')
output.write("parameter search results\n")
                
print("Hyperparameters and their accuracies:")
print("parameter search for epoch 5")
print("parameters: depth, batch size, optizer, learning rate: train accuracy, validation accuracy, test accuracy")

output.write("Hyperparameters and their accuracies:\n")
output.write("parameter search for epoch 5\n")
output.write("parameters: depth, batch size, optizer, learning rate: train accuracy, validation accuracy, test accuracy\n")

output.flush()

for depth in network_depths:
    for batch_size in batch_sizes:
        for opt in optimizers:
            for learn in learning_rates:
                if depth == 18:
                    model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
                    weights = models.ResNet18_Weights.DEFAULT
                elif depth == 34:
                    model = models.resnet34(weights=models.ResNet34_Weights.IMAGENET1K_V1)
                    weights = models.ResNet34_Weights.DEFAULT
                elif depth == 50:
                    model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
                    weights = models.ResNet50_Weights.DEFAULT

                preprocess = weights.transforms()
                transform = transforms.Compose([
                    preprocess
                    ])
                train_dataset = CassavaDataset(train_df, train_dir, transform=transform)
                validation_dataset = CassavaDataset(val_df, train_dir, transform=transform)
                test_dataset = CassavaDataset(test_df, train_dir, transform=transform)

                # Create the data loader
                train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                val_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True)
                test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


                # Linear Layers
                model.fc = torch.nn.Sequential(
                    torch.nn.Linear(model.fc.in_features, 512),
                    torch.nn.ReLU(),
                    torch.nn.Linear(512, 256),
                    torch.nn.ReLU(),
                    torch.nn.Linear(256, NUM_LABELS)
                )

                model.to(device)

                criterion = torch.nn.CrossEntropyLoss()
                if opt == 'adam':
                    optimizer = torch.optim.Adam(model.parameters(), lr=learn)
                elif opt == 'SGD':
                    optimizer = torch.optim.SGD(model.parameters(), lr=learn, momentum=0.9)
                
                # Train and validate the model
                train_acc, val_acc, test_acc = train_and_validate(model, train_loader, val_loader,test_loader, optimizer, criterion)
                
                hyperparams_accuracies[(depth, batch_size, opt, learn)] = (train_acc, val_acc, test_acc)
                print(f"finish training/validate for parameters:", (depth, batch_size, opt, learn))
                output.write(f"{(depth, batch_size, opt, learn)}: {(train_acc, val_acc, test_acc)}\n")
                output.flush()
                save_model(model,"resnet" + str(depth) + "_batch_" + str(batch_size)+"_opti_" + str(opt)+"_lr_"+str(int(learn*10000)))
    
for params, accuracy in hyperparams_accuracies.items():
    print(f"{params}: {accuracy}")
    # output.write(f"{params}: {accuracy}\n")

output.close()

Hyperparameters and their accuracies:
parameter search for epoch 5
parameters: depth, batch size, optizer, learning rate: train accuracy, validation accuracy, test accuracy
